# Query with SQLite3

### Import modules and create database

In [3]:
import sqlite3
import pandas as pd
import os

try:
    os.remove('airline3.db')
except OSError:
    pass

In [4]:
conn = sqlite3.connect('airline3.db')

### Tables

In [6]:
airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")
planes = pd.read_csv("plane-data.csv")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

5029

In [ ]:
# Process flight data
years = range(2004, 2007)
info = ['CRSArrTime', 'CRSDepTime', 'ArrDelay', 'TailNum', 'Diverted', 'UniqueCarrier', 'Dest']

In [22]:
for year in range(2004, 2007):
    ontime = pd.read_csv(str(year)+".csv", encoding = "ISO-8859-1")
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)


### Queries

In [ ]:
#Open connection to database
c = conn.cursor()

Query 1: Which of the following airplanes has the lowest associated average departure delay (excluding cancelled and diverted flights)?

In [24]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

print(c.fetchone()[0], "was the lowest associated average departure delay.")

182A was the lowest associated average departure delay.


Query 2: Find the city that has the highest number of inbound flights (excluding canceled flights)

In [26]:
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

print(c.fetchone()[0], "has the highest number of inbound flights (excluding canceled flights)")

Chicago has the highest number of inbound flights (excluding canceled flights)


Query 3: Find the carrier that has the highest number of canceled flights

In [27]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC
''')

print(c.fetchone()[0],"has the highest number of canceled flights")

American Airlines Inc. has the highest number of canceled flights


Query 4: Find the carrier that has the highest number of canceled flights, relative to their number of total flights

In [28]:
c.execute('''
SELECT
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')
print(c.fetchone()[0], "has the highest number of canceled flights, relative to their number of total flights")

Delta Air Lines Inc. has the highest number of canceled flights, relative to their number of total flights


In [ ]:
#Close database connection
conn.close()